# NLP - Word Embeddings - Pascal Thürig

## Introduction
Starting point for this project is the following key requirements:
1. Use the BoolQ Dataset from Hugging Face
2. Use pre-trained model for word embeddings (word2vec, GloVe or fastText)
3. Train a 2-layer classifier with ReLU non-linearity

In this project I will be using pre-trained embeddings from word2vec and a simple 2-layer neural network to do the reading comprehension task on the BoolQ dataset.
I will document every decision made, from preprocessing to model training and evaluation. The goal is to classify each BoolQ question-answer pair as either 'Yes' or 'No'.

## TLDR; Here are the key decisions and justifications:
- BoolQ Dataset: Provided by Project berief
- Task: Classify BoolQ questions as either "yes" or "no" using pre-trained embeddings and a simple neural network
- Pre-trained embeddings: word2vec - Google News 300; for simplicity and already have a bit of experience with it
- Model: 2-layer NN with ReLU activation: Provided by Project brief
- Tokenizing: Yes, using a subword tokenizer.
- Lowercasing: Yes, all text will be lowercased.
- Stemming: No, stemming will not be applied.
- Lemmatizing: No, lemmatizing is not used initially but could be tried later.
- Stopword removal: No, stopwords are not removed to retain key information.
- Removal of other words: No, no other word removal is planned.
- Format cleaning: No further cleaning required, the dataset is already clean.
- Truncation: Yes, input text is truncated to a maximum of 512 tokens.
- Feature selection: None, relying on word2vec embeddings directly.
- Input format: Tokenized and padded sequences of word2vec embeddings.
- Label format: Binary (1 for "yes", 0 for "no").
- train/valid/test splits: 66% train, 8% validation, 26% test.
- Padding: Yes, sequences are padded for uniform input length.
- Embedding: Pre-trained word2vec embeddings are used for simplicity.
- Planned correctness tests: Shape consistency checks, binary label correctness, and validation of truncation and padding.
- Hyperparameters:
    - Learning Rate: 1e-2 – 1e-5
    - Batch Size: 16 - 64 (choosing maximum possible that my GPU can handle)
    - Epochs 10 - 50 in 5-/10-step increments
    - Hidden size: 64 - 512 
    - Early Stopping: Patience of 3 - 10 Epochs of non-improvement (depending on total epoch number)
- Evaluation: Accuracy and F1-Score: Accuracy for general performance and F1 to handle class imbalances
- Error Analysis: Investigating False Positives and False negatives to understand where the model fails

## Setup
Importing necessary libraries:
- datasets
- gensim
- transformers
- numpy
- torch
- wandb
- sklearn

First up the BoolQ dataset is loaded

For easy access during experiments I like to define the hyperparameters at the top of my notebooks

In [9]:
# Hyperparameters:


Now the pre-trained embeddings from word2vec

In [ ]:
# (down-)load word2vec - word2vec-google-news-300

## Preprocessing

The BoolQ data will be processed in the following way:
1.  Tokenizing: the input questions and passages using a subword tokenizer
2.  Lowercasing: the text for simplicity and to reduce the total vocabulary size
3.  Stemming: No, will not stem the words as to not lose information
4.  Lemmatizing: No, will try if it improves performance
5.  Stopword removal: No, will not be removed to not lose potentially critical information [research](https://datascience.stackexchange.com/questions/31048/pros-cons-of-stop-word-removal)
6.  Removal of other words: No, will not be removing any other words
7.  Format cleaning: The dataset is already sufficiently clean, it shouldn't impact performance
8.  Truncation: the input text is truncated to a maximum of 512 tokens
9.  Feature selection: Not applicable as we focus on raw text as input and leveraging the pre-trained word embeddings no further feature extraction is needed.
10. Input format: Will take the form of the tokenized and padded sequences of word embeddings
11. Label format: Binary labels "yes" or "no"
12. train/valid/test splits: Prerequisite to project (66/8/26)
13. Padding: the sequences is padded to ensure all inputs have the same length in each batch
14. Embedding: Using word2vec, solely for simplicity as I already know it.
15. Planned correctness tests: Check for shape mismatches between tokenized text and word embeddings. - Ensure that input sequences are properly truncated and padded. - Verify that binary labels are correctly assigned and match the expected outputs.


1. Preprocess text (lowercasing)

2. Tokenize with AutoTokenizer from Hugging Face

In [11]:
# tokenize w/ AutoTokenizer.from_pretrained("bert-base-uncased")

3. Truncate or add padding

4. embed tokens using word2vec (word2vec-google-news-300)

5. Create a custom BoolQ dataset class to:
    - get the data into a compatible format for the pyTorch dataloader.
    - organize question-answer pairs and apply the preprocessing pipeline.
    - easily batch, shuffle, and load the data during training.

In [12]:
# class BoolQDataset(dataset):

6. Dataloaders as required by pyTorch

7. Initialize weights and biases for experiment tracking

## Model

The model architecture for this project is already fixed in the project brief as follows:
- **Network Architecture:** 2-Layer with ReLu non-linearity.
- **Loss / Optimizer:** Loss: CrossEntropyLoss / Optimizer: Adam (potentially trying SGD with or without momentum in experiments)
- **Experiments to run**: Mentioned in Training section below
- **Number of training runs**: Will depend on number of experiments
- **Checkpointing / Early stopping:** 3 - 10 epochs of non-improvement of the validation loss
- **Planned correctness tests:** Shape and Dimension consistency tests, Gradient Check, Sanity Check & Prediction Testd

1. Creating the neural network class:

2. Create instance of model and move it to the GPU

3. Loss (nn.CrossEntropyLoss) and optimizer (optim.Adam)

4. Training loop

5. Evaluation function

## Training
Train the model with the following different hyperparameters:
- Learning rate: 1e-2 – 1e-5
- Batch size: 16 - 64
- Epochs: 10 - 50
- Hidden size: 64 - 512
- Early Stopping: Patience of 3 - 10 Epochs of non-improvement


## Evaluation
The model will be evaluated for the key metrics of:
- Accuracy
- Precision
- Recall
- F1 Score

The results will be averaged using micro averaging because I care about the total number of correct prediction regardless of the class ("yes" or "no"). 

Errors will be evaluated by making a confusion matrix and giving me the distribution of ture positives, false positives, true negatives and false negatives. Helping me figure out where the model is making most of it's mistakes.

## Finish the WandB run
Closing the WandB run

## Interpretation

To set concrete expectations for my model I take into account a couple of key benchmarks:
- **Accuracy:** Given the task of binary classification an accuracy of ~50% can be achieved with random guesses.
    - Expecting my model to hit an accuracy of ~60-75%.
- **F1 Score:** For this dataset I expect the F1 score to be similar to the accuracy of ~60-75%